###### * Base Source: https://www.kaggle.com/wangsg/a-self-attentive-model-for-knowledge-tracing
* My First Work: https://www.kaggle.com/leadbest/sakt-self-attentive-knowledge-tracing-submitter

1. Version 1: State Updates -> LB 0.765
2. Version 3: Random Selection of User Interactions -> LB 0.768
3. Version 6: Small Optimization -> LB 0.771?

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
#HDKIM
MAX_SEQ = 160
#HDKIMHDKIM

## Load data

In [ ]:
%%time
dtype = {'row_id': 'int64','timestamp':'int64', 
         'user_id':'int32' ,
         'content_id':'int16',
         'content_type_id':'int8',
         'answered_correctly':'int8','prior_question_elapsed_time': 'float32'}

data_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', usecols=[0,1, 2, 3, 4, 7,8], dtype=dtype)
data_df.head()

In [ ]:
data_df = data_df[data_df.content_type_id == False] #ojo con esto que yp lo tengo como 0 y 1 el flag

#arrange by timestamp
data_df = data_df.sort_values(['timestamp'], ascending=True).reset_index(drop = True)

## Preprocess

In [ ]:
skills = data_df["content_id"].unique()
n_skill = len(skills)
print("number skills", len(skills))

group = data_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values))

del data_df
gc.collect()

In [ ]:
#HDKIM
import random
random.seed(1)
#HDKIMHDKIM

In [ ]:
from sklearn.model_selection import StratifiedKFold,train_test_split






In [ ]:
#    unique_indexes= y_train.groupby('user_id_x').mean()

#     #aux_df = pd.concat([X_train["variable"],y_train],axis=1)

#   #  for fold,(train_index, test_index) in enumerate(mkf.split(unique_indexes,unique_indexes)):
     
#     for fold,(train_index, val_index) in enumerate(mkf.split(unique_indexes,pd.qcut(unique_indexes,5,labels = [i for i in range(5)],duplicate='drop'))):

In [ ]:
def reshape_to_accomodate_max_len(x,cut_len =160):
    
    # This assumes that you have a 2-dimensional array
    #zeros = np.zeros(shape, dtype=np.int32)
   # zeros[:sigma.shape[0], :sigma.shape[1]] = sigma
    
    
    if x.shape[0] >cut_len:
        cuts = np.arange(0,x.shape[0],cut_len) if  ( x.shape[0] % cut_len ) else  np.arange(0,x.shape[0],cut_len).tolist() +[x.shape[0]]
        return np.array_split(x,cuts)
    else:
        return [x]

In [ ]:
def get_lag_time(df):
    
    df.set_index('row_id',inplace = True)

    
    
    
    return df['timestamp'].diff() - df['prior_question_elapsed_time']

In [ ]:
#lag_time
res_lag_time = data_df[['row_id','user_id','timestamp','prior_question_elapsed_time']].groupby('user_id').apply(get_lag_time )

In [ ]:
prior_question_elapsed_time

#prior_question_elapsed_time is made with a shift from the prior

In [ ]:
%%time
group = data_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: 
            reshape_to_accomodate_max_len(r[['content_id', 'answered_correctly']].values,cut_len=640)).explode().apply(lambda x: np.array(x).transpose())


del data_df
gc.collect()

In [ ]:
unique_indexes= group.apply(lambda x: len(x[0]) )


group_train,group_val = train_test_split(group,test_size=0.1,stratify = pd.qcut(unique_indexes,5,labels = [i for i in range(5)]),random_state=14)



In [ ]:
group_train.head()

In [ ]:
group_train.reset_index(drop=True,inplace=True)
group_val.reset_index(drop=True,inplace=True)

In [ ]:
group = group.groupby('user_id').tail(1)

In [ ]:
plt.hist(group.apply(lambda x:len(x[0])))

In [ ]:
group.loc[(group.apply(lambda x:len(x[0])) ==0)]

In [ ]:
group = group.loc[(group.apply(lambda x:len(x[0])) !=0)]

In [ ]:
import pickle as pk

pk.dump(group,open('group.pkl','wb'))

In [ ]:
del group
gc.collect()

In [ ]:
class SAKTDataset(Dataset):
    def __init__(self, group, n_skill, max_seq=MAX_SEQ): #HDKIM 100
        super(SAKTDataset, self).__init__()
        self.max_seq = max_seq
        self.n_skill = n_skill
        self.samples = group

        #         self.user_ids = [x for x in group.index]
        self.user_ids = []
        for user_id in group.index:
            q, qa = group[user_id]
            if len(q) < 2: #HDKIM 10
                 continue
            self.user_ids.append(user_id)

        #HDKIM Memory reduction
        #if len(q)>self.max_seq:
        #    group[user_id] = (q[-self.max_seq:],qa[-self.max_seq:])

    def __len__(self):
        return len(self.user_ids)
        #return len(self.samples)

    def __getitem__(self, index):
        user_id = self.user_ids[index]
        q_, qa_ = self.samples[user_id]
        #q_, qa_ = self.samples.iloc[index]
        seq_len = len(q_)

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        
        if seq_len >= self.max_seq:
            #HDKIM
            if random.random()>0.02:
                start = random.randint(0,(seq_len-self.max_seq))
                end = start + self.max_seq
                q[:] = q_[start:end]
                qa[:] = qa_[start:end]
            else:
                #HDKIMHDKIM
                q[:] = q_[-self.max_seq:]
                qa[:] = qa_[-self.max_seq:]
        else:
            #HDKIM
            if random.random()>0.02:
                #HDKIMHDKIM
                start = 0
                end = random.randint(2,seq_len)
                seq_len = end - start
                q[-seq_len:] = q_[0:seq_len]
                qa[-seq_len:] = qa_[0:seq_len]
            else:
                #HDKIMHDKIM
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_

        
        target_id = q[1:]
        label = qa[1:]

        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[:-1].copy()
        x += (qa[:-1] == 1) * self.n_skill

        return x, target_id, label

In [ ]:
train_dataset = SAKTDataset(group_train, n_skill)
val_dataset = SAKTDataset(group_val, n_skill)

#dataloader = DataLoader(dataset, batch_size=2048, shuffle=True, num_workers=8)

item = train_dataset.__getitem__(5)
# print(item[0])
# print(item[1])
# print(item[2])

## Define model

In [ ]:
import math
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens
        in the sequence. The positional encodings have the same dimension as
        the embeddings, so that the two can be summed. Here, we use sine and cosine
        functions of different frequencies.
    .. math::
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        #position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
       # position =torch.repeat_interleave(torch.arange(0, max_len  // tokens_per_word , dtype=torch.float), tokens_per_word).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """

        return self.pe[:x.size(0), :]

In [ ]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class SAKTModel(nn.Module):
    def __init__(self, n_skill, max_seq=MAX_SEQ, embed_dim=128): #HDKIM 100->MAX_SEQ
        super(SAKTModel, self).__init__()
        self.n_skill = n_skill
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(2*n_skill+1, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim)

        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=0.2)

        self.dropout = nn.Dropout(0.2)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids):
        device = x.device        
        x = self.embedding(x)
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)

        pos_x = self.pos_embedding(pos_id)
        x = x + pos_x

        e = self.e_embedding(question_ids)

        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = e.permute(1, 0, 2)
        att_mask = future_mask(x.size(0)).to(device)
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
        att_output = self.layer_normal(att_output + e)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1), att_weight

In [ ]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class PosSAKTModel(nn.Module):
    def __init__(self, n_skill, max_seq=MAX_SEQ, embed_dim=128): #HDKIM 100->MAX_SEQ
        super(SAKTModel, self).__init__()
        self.n_skill = n_skill
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(2*n_skill+1, embed_dim)
        
        #self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
        self.pos_embedding = PositionalEncoding(max_len=max_seq-1,d_model=embed_dim)   #nn.Embedding(max_seq-1, embed_dim)
        
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim)

        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=0.2)

        self.dropout = nn.Dropout(0.2)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids):
        device = x.device        
        x = self.embedding(x)
        
       # print(x.size(0),x.size(1),x.size(2))
       # print(x.size(1))
        
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)

        pos_x = self.pos_embedding(pos_id)
        
        x = x + pos_x

        e = self.e_embedding(question_ids)

        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = e.permute(1, 0, 2)
        att_mask = future_mask(x.size(0)).to(device)
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
        att_output = self.layer_normal(att_output + e)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1), att_weight

In [ ]:
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)

def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)


class SAKTmodule(nn.Module):
    def __init__(self,  embed_dim=128,num_heads=8,dropout=0.1): #HDKIM 100->MAX_SEQ
        super(SAKTmodule, self).__init__()
        
       # self.att_mask = att_mask

        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout= dropout)
        self.self_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout= dropout)

        
        
        self.layer_normal_self1 = nn.LayerNorm(embed_dim) 
        self.layer_normal_self2 = nn.LayerNorm(embed_dim) 
        
        self.layer_normal1 = nn.LayerNorm(embed_dim) 
        self.layer_normal2 = nn.LayerNorm(embed_dim) 

        self.ffn1 = FFN(embed_dim)
        self.ffn2 = FFN(embed_dim)
        
       
    
    def forward(self, x, e,att_mask):
       

        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = e.permute(1, 0, 2)
        
        selfatt_output, selfatt_mask = self.self_att(e, e, e, attn_mask= att_mask)
        
       
        e = self.layer_normal_self1(selfatt_output + e) #se suma esto ahora?
        
        e = e.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

         
        e1 = self.ffn1(e)
        
        e = self.layer_normal_self2(e1 + e)
        
        e = e.permute(1, 0, 2)
        
        att_output, att_weight = self.multi_att(e, x, x, attn_mask= att_mask)
        
        e = self.layer_normal1(att_output + e)
        
        e = e.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn2(e)
        x = self.layer_normal2(x + e)
        
        
        

        return x, e ,att_mask
    
    








class SSAKTModel(nn.Module):
    def __init__(self, n_skill, max_seq=MAX_SEQ, embed_dim=128): #HDKIM 100->MAX_SEQ
        super(SSAKTModel, self).__init__()
        self.n_skill = n_skill
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(2*n_skill+1, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim)

        self.SAKT1 = SAKTmodule(embed_dim=embed_dim)
        self.SAKT2 = SAKTmodule(embed_dim=embed_dim)
        self.SAKT3 = SAKTmodule(embed_dim=embed_dim)

        
        
        self.dropout = nn.Dropout(0.2)
        
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids):
        device = x.device        
        x = self.embedding(x)
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)

        pos_x = self.pos_embedding(pos_id)
        x = x + pos_x

        e = self.e_embedding(question_ids)
        
        x = x.permute(1, 0, 2)
        
        att_mask = future_mask(x.size(0)).to(device)
        
        x = x.permute(1, 0, 2)
        
        x, e,att_mask = self.SAKT1(x, e,att_mask)
        x, e,att_mask = self.SAKT2(x, e,att_mask)
        x, e,att_mask= self.SAKT3(x, e,att_mask)
    
        x = self.pred(x)
        
        

        return x.squeeze(-1),att_mask
    
    
    
    
    

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = SAKTModel(n_skill, embed_dim=128)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.99, weight_decay=0.005)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#criterion = nn.BCEWithLogitsLoss()

#model.to(device)
#criterion.to(device)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    
    tbar = tqdm(dataloader)
    
    for item in tbar:
        optimizer.zero_grad()
        
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()

      
        output, _ = model(x, target_id)
        
        loss = loss_fn(output, label)
        loss.backward()
        
        optimizer.step()
        
        if not  scheduler.__class__ ==  torch.optim.lr_scheduler.ReduceLROnPlateau:
            scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, scheduler, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    labels = []
    
    tbar = tqdm(dataloader)
    
    for item in tbar:
       
        
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()

      
        output, _ = model(x, target_id)
        
        loss = loss_fn(output, label)
        
        
        output = torch.sigmoid(output)
        
         
        output = output[:, -1]
        label = label[:, -1] 
        
        
        labels.extend(label.view(-1).data.cpu().numpy())
        valid_preds.extend(output.view(-1).data.cpu().numpy())
       
        
        #valid_preds.append(output.sigmoid().detach().cpu().numpy())
       # labels.append(label.detach().cpu().numpy()[0])
        
        if scheduler.__class__ ==  torch.optim.lr_scheduler.ReduceLROnPlateau:
            scheduler.step(final_loss)
    
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    auc = roc_auc_score(labels, valid_preds)
    
    return final_loss,valid_preds,auc
    

def inference_fn(model, dataloader, device):
    model.eval()
    outs = []
    
    tbar = tqdm(dataloader)
    
    for item in tbar:
        optimizer.zero_grad()
        
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()

        with torch.no_grad():
            output, _ = model(x, target_id)
            
        output = torch.sigmoid(output)
        output = output[:, -1]
        
        outs.extend(output.data.cpu().numpy())
    
    return outs


def infer_features_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for item in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs,mode='get_features')
        
#         print(len(outputs))
        
#         for i in range(len(outputs)):
#             print(outputs[i].shape)
            
#         print(torch.cat(outputs,axis=1).shape)
        
        preds.append(torch.cat(outputs,axis=1).detach().cpu().numpy())
        
        
        
    preds = np.concatenate(preds)
    
    return preds
   
    

In [ ]:
def accuracy_score():
    
    pred = (torch.sigmoid(output) >= 0.5).long()

    num_corrects += (pred == label).sum().item()
    num_total += len(label)

    auc = roc_auc_score(labels, outs)
    acc = num_corrects / num_total
    


In [ ]:
#chequear q quc no sea solo en ultima pregunta

In [ ]:

EPOCHS = 40

LR = 6e-3

BATCH_SIZE = 256

EARLY_STOPPING_STEPS = 5

EARLY_STOP = False

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    #os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler



In [ ]:
class NoamLR(_LRScheduler):
    """
    Implements the Noam Learning rate schedule. This corresponds to increasing the learning rate
    linearly for the first ``warmup_steps`` training steps, and decreasing it thereafter proportionally
    to the inverse square root of the step number, scaled by the inverse square root of the
    dimensionality of the model. Time will tell if this is just madness or it's actually important.
    Parameters
    ----------
    warmup_steps: ``int``, required.
        The number of steps to linearly increase the learning rate.
    """
    def __init__(self, optimizer, warmup_steps):
        self.warmup_steps = warmup_steps
        super().__init__(optimizer)

    def get_lr(self):
        last_epoch = max(1, self.last_epoch)
        scale = self.warmup_steps ** 0.5 * min(last_epoch ** (-0.5), last_epoch * self.warmup_steps ** (-1.5))
        return [base_lr * scale for base_lr in self.base_lrs]


In [ ]:
    
def train_pipeline(train_dataset,valid_dataset, seed,fold,verbose=False,**kwargs):  
    
    seed_everything(seed)
    
   
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
   
    model = SSAKTModel(n_skill, embed_dim=128)
    
    model.to(DEVICE)
    
    #initialize_from_past_model(model, f"../results/original_torch_moa_smoothed_lrplateau_5_folds_AUX_SEED{seed}_FOLD{fold}.pth")#,freeze_first_layer=True)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    #optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer, pct_start=0.1, div_factor=1e3, 
     #                                         max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    
    scheduler = NoamLR(optimizer,12000) #optim.lr_scheduler.ReduceLROnPlateau(optimizer,patience=3)
    
    
    
    loss_val = nn.BCEWithLogitsLoss().to(DEVICE)
    loss_tr =  nn.BCEWithLogitsLoss().to(DEVICE)
    
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    #todo el guardado de los resultados se puede mover a kfold que si tiene info de los indices
    #oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    best_auc = 0
    
    
    
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        if verbose:
            print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}, lr:{scheduler.get_last_lr()[0]}")
        valid_loss, valid_preds ,auc = valid_fn(model,scheduler, loss_val, validloader, DEVICE)
        if verbose:
            print(f"FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}, auc: {auc} ")

    #         if valid_loss < best_loss:

    #             best_loss = valid_loss
    #             oof = valid_preds
    #             torch.save(model.state_dict(), "SAKT-HDKIM.pt")

        if auc > best_auc:

            best_auc = auc
            oof = valid_preds
            torch.save(model.state_dict(), "SAKT-HDKIM.pt")



       #     torch.save(model.state_dict(), f"../results/{exp_name}_SEED{seed}_FOLD{fold}.pth")

        elif(EARLY_STOP == True):

            early_step += 1
            if (early_step >= early_stopping_steps):
                break

    
    #--------------------- PREDICTION---------------------
   
  #  testdataset = TestDataset(X_test)
  #  testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
#     model = Model(
#          num_features= X_train.shape[1] ,
#         num_targets=  y_train.shape[1],
#         hidden_size=hidden_size,**kwargs
#     )
    
#     model.load_state_dict(torch.load(f"../results/FOLD{fold}_{exp_name}.pth"))
   # model.to(DEVICE)
    
    #predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    
    #predictions = inference_fn(model, testloader, DEVICE)
    
    return oof


In [ ]:
 oof = train_pipeline(train_dataset,val_dataset,42,fold=0,verbose=True)

def train_epoch(model, train_iterator, optim, criterion, device="cpu"):
    model.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []
    
    
    
    tr_loss = train_fn()
    
    

    tbar = tqdm(train_iterator)
    for item in tbar:
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()

        optim.zero_grad()
        output, atten_weight = model(x, target_id)
        loss = criterion(output, label)
        loss.backward()
        optim.step()
        train_loss.append(loss.item())
        
        
        
        trI
       
        output = output[:, -1]
        label = label[:, -1] 
        
        
        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

        #accuracy
        pred = (torch.sigmoid(output) >= 0.5).long()
        
        num_corrects += (pred == label).sum().item()
        num_total += len(label)

        tbar.set_description('loss - {:.4f}'.format(loss))

    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.mean(train_loss)

    return loss, acc, auc

epochs = 35 #HDKIM 20
for epoch in range(epochs):
    
    loss, acc, auc = train_epoch(model, dataloader, optimizer, criterion, device)
    print("epoch - {} train_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, loss, acc, auc))

In [ ]:
import gc
del train_dataset,val_dataset
gc.collect()

## Test

In [ ]:
group = pk.load(open('group.pkl','rb'))

In [ ]:
class TestDataset(Dataset):
    def __init__(self, samples, test_df, skills, max_seq=MAX_SEQ): #HDKIM 100
        super(TestDataset, self).__init__()
        self.samples = samples
        self.user_ids = [x for x in test_df["user_id"].unique()]
        self.test_df = test_df
        self.skills = skills
        self.n_skill = len(skills)
        self.max_seq = max_seq

    def __len__(self):
        return self.test_df.shape[0]

    def __getitem__(self, index):
        test_info = self.test_df.iloc[index]

        user_id = test_info["user_id"]
        target_id = test_info["content_id"]

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)

        if user_id in self.samples.index:
            q_, qa_ = self.samples[user_id]
            
            seq_len = len(q_)

            if seq_len >= self.max_seq:
                q = q_[-self.max_seq:]
                qa = qa_[-self.max_seq:]
            else:
                q[-seq_len:] = q_
                qa[-seq_len:] = qa_          
        
        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[1:].copy()
        x += (qa[1:] == 1) * self.n_skill
        
        questions = np.append(q[2:], [target_id])
        
        return x, questions

In [ ]:
import riiideducation

env = riiideducation.make_env()
iter_test = env.iter_test()

In [ ]:
model = SSAKTModel(n_skill, embed_dim=128)

model.load_state_dict(torch.load(f"SAKT-HDKIM.pt",map_location=device))
model.to(device)

In [ ]:
import psutil
model.eval()

#HDKIM
prev_test_df = None
#HDKIMHDKIM

for (test_df, sample_prediction_df) in tqdm(iter_test):
    #HDKIM
    if (prev_test_df is not None) & (psutil.virtual_memory().percent<90):
        print(psutil.virtual_memory().percent)
        prev_test_df['answered_correctly'] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prev_test_df = prev_test_df[prev_test_df.content_type_id == False]
        prev_group = prev_test_df[['user_id', 'content_id', 'answered_correctly']].groupby('user_id').apply(lambda r: (
            r['content_id'].values,
            r['answered_correctly'].values))
        for prev_user_id in prev_group.index:
            prev_group_content = prev_group[prev_user_id][0]
            prev_group_ac = prev_group[prev_user_id][1]
            if prev_user_id in group.index:
                group[prev_user_id] = (np.append(group[prev_user_id][0],prev_group_content), 
                                       np.append(group[prev_user_id][1],prev_group_ac))
 
            else:
                group[prev_user_id] = (prev_group_content,prev_group_ac)
            if len(group[prev_user_id][0])>MAX_SEQ:
                new_group_content = group[prev_user_id][0][-MAX_SEQ:]
                new_group_ac = group[prev_user_id][1][-MAX_SEQ:]
                group[prev_user_id] = (new_group_content,new_group_ac)

    prev_test_df = test_df.copy()
 
    #HDKIMHDKIM
    
    test_df = test_df[test_df.content_type_id == False]
                
    test_dataset = TestDataset(group, test_df, skills)
    test_dataloader = DataLoader(test_dataset, batch_size=51200, shuffle=False)
    
    outs = []

    for item in tqdm(test_dataloader):
        x = item[0].to(device).long()
        target_id = item[1].to(device).long()

        with torch.no_grad():
            output, att_weight = model(x, target_id)
        
        
        output = torch.sigmoid(output)
        output = output[:, -1]

        # pred = (output >= 0.5).long()
        # loss = criterion(output, label)

        # val_loss.append(loss.item())
        # num_corrects += (pred == label).sum().item()
        # num_total += len(label)

        # labels.extend(label.squeeze(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())
        
    test_df['answered_correctly'] =  outs
    
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])